In [9]:
from PdfCleaner import PdfCleaner
from os import listdir
from os.path import isfile, join
import pdfplumber
import json
import pandas as pd
import numpy as np
import requests
from pymongo import MongoClient


class Transcripts:
    def __init__(self, ticker):
        self.ticker = ticker
        self.path = "/Users/emilywang/Desktop/DS3500-Final-Project-main-2/transcripts/"+ticker+"_transcripts/"

    def read_files(self):
        return [f for f in listdir(self.path) if isfile(join(self.path, f))]

    def create_dct(self):
        lst_files = self.read_files()
        lst_cleaned = []
        for file in lst_files:
            file_path = self.path + file
            txt = PdfCleaner(file_path)
            date = file[:8]
            txt_cleaned = txt.clean_stopwords_punctuation()
            dct_cleaned = {'name': self.ticker, 'date': date, 'transcript': txt_cleaned}
            lst_cleaned.append(dct_cleaned)
        return lst_cleaned

class Database:
    def __init__(self):
        client = MongoClient()
        client.drop_database('transcripts')
        self.db = client.transcripts

    def store_data(self, tickers_lst):
        # client = MongoClient()
        # client.drop_database('transcripts')
        # db = client.transcripts
        for t in tickers_lst:
            store = Transcripts(t)
            transcript = store.create_dct()
            self.db.transcript.insert_many(transcript)
            print(t + " transcripts stored successfully")
        return self.db

# def main():

    


# if __name__ == '__main__':
#     main()

In [12]:
#  tickers = ['APPL', 'MSFT']
tickers = ['APPL']
    # works: APPL, MSFT
    # doesn't work: FB, GOOGL, NFLX, TSLA, ADBE, CMCSA, COST, AMZN

    # 'APPL', 'MSFT', 'FB', 'GOOGL', 'NFLX', 'TSLA', 'ADBE', 'CMCSA', 'COST', 'AMZN'
    # client = MongoClient()
    # client.drop_database('transcripts')
    # db = client.transcripts


    # for t in tickers:
    #     store = Transcripts(t)
    #     transcript = store.create_dct()
    #     # print(transcript)
    #     db.transcript.insert_many(transcript)
    #     print(t + " transcripts stored successfully")

    # db.transcript.find() # client.transcripts.transcript.find()

database = Database()
db = database.store_data(tickers)
all_transcripts = db.transcript.find()
for transcript in all_transcripts:
    print(transcript)

APPL transcripts stored successfully
{'_id': ObjectId('61a2cdbb72d6dc36b0ae2348'), 'name': 'APPL', 'date': '20170801', 'transcript': 'Company Name Apple Inc Company Ticker AAPL US Equity Date 20170801 Q3 2017 Earnings Call Company Participants Luca Maestri Chief Financial Oﬃcer  Senior Vice President L Nancy Paxton Senior Director Investor Relations Treasury A N Timothy Donald Cook Chief Executive Oﬃcer  Director I F Other Participants Amit Daryanani Analyst Antonio M Sacconaghi Analyst Brian J White Analyst Kathryn Lynn Huberty Analyst Kulbinder S Garcha Analyst Michael J Olson Analyst Shannon S Cross Analyst Steven Milunovich Analyst MANAGEMENT DISCUSSION SECTION t Operator p r Good day everyone welcome Apple Incorporated Third Quarter Fiscal Year c 2017 Earnings Release Conference Call Todays recorded s n At time opening remarks introductions I like turn r Nancy Paxton Senior Director Investor Relations Please ahead maam T g Nancy Paxton BIO 1779050 GO r e Thank you Good afternoon t

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


''' Query to test on all APPLE transcripts '''
apple = db.transcript.find({"name":"APPL"})

    
data = {}
for transcript in apple:
    data[transcript["name"]+" "+transcript["date"]] = transcript["transcript"]
#     df = pd.DataFrame.from_dict(data, orient ='index')


for name, info in data.items():
    ''' Use countvectorizer to convert pdf text to a matrix of token counts '''
    text = [info]
    vect = CountVectorizer()
    vect.fit(text)
#     print("vocabulary: \n", vect.vocabulary_)


    ''' Use bag of words to count how many times each word appears in pdf '''
    bag_of_words = vect.transform(text)
#     print(bag_of_words)

    # number of times each unique word appears in this text
    bag_of_words_arr = bag_of_words.toarray()
#     print(bag_of_words_arr)
    feature_names = vect.get_feature_names()
#     print(feature_names)

    ''' Display occurrences in dataframe '''
    bow_df = pd.DataFrame(bag_of_words_arr, columns = feature_names)
    bow_df["Transcript (Company & Date)"] = name
    bow_df.set_index("Transcript (Company & Date)", inplace=True)

    ''' Convert bag of word values into TFIDF values (based on the number of occurrences for each word) '''

#     vect = TfidfVectorizer(ngram_range = (1, 2)).fit(df[0])
#     bag_of_words = vect.transform(text)
#     feature_names = vect.get_feature_names()
    
#     tfidf_df = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)
#     tfidf_df[name] = text
#     tfidf_df.set_index(name, inplace=True)

#     tfidf_df.head()
    
    
    tfidfconverter = TfidfTransformer()
    X = tfidfconverter.fit_transform(bag_of_words_arr).toarray()

    
    tfidf_df = pd.DataFrame(X, columns = feature_names)
    tfidf_df["Transcript (Company & Date)"] = name
    tfidf_df.set_index("Transcript (Company & Date)", inplace=True)

    ''' Show number of occurences of each word in one dataframe and tfidf (weighted) values of each word in another
    dataframe for each pdf '''
    display(bow_df)
    display(tfidf_df)

,063,10,100,1000,105inch,108b,10th,11,114mm,117b,...,wwdc,year,yearonyear,years,yes,yet,you,your,yovery,ytd
Transcript (Company & Date),,,,,,,,,,,,,,,,,,,,,
APPL 20170801,1,2,2,1,2,1,1,10,1,1,...,5,56,2,7,2,1,14,1,12,28


,063,10,100,1000,105inch,108b,10th,11,114mm,117b,...,wwdc,year,yearonyear,years,yes,yet,you,your,yovery,ytd
Transcript (Company & Date),,,,,,,,,,,,,,,,,,,,,
APPL 20170801,0.003415,0.00683,0.00683,0.003415,0.00683,0.003415,0.003415,0.034148,0.003415,0.003415,...,0.017074,0.19123,0.00683,0.023904,0.00683,0.003415,0.047807,0.003415,0.040978,0.095615


,073,10,100,1026,10k,10q,11,1128,1142,115,...,ﬁnd,ﬁrst,ﬁscal,ﬁtness,ﬁve,ﬂat,ﬂattodown,ﬂow,ﬂuctuations,ﬂuid
Transcript (Company & Date),,,,,,,,,,,,,,,,,,,,,
APPL 20180731,1,7,6,1,1,1,2,1,1,1,...,1,8,3,2,2,5,1,1,1,1


,073,10,100,1026,10k,10q,11,1128,1142,115,...,ﬁnd,ﬁrst,ﬁscal,ﬁtness,ﬁve,ﬂat,ﬂattodown,ﬂow,ﬂuctuations,ﬂuid
Transcript (Company & Date),,,,,,,,,,,,,,,,,,,,,
APPL 20180731,0.004184,0.029286,0.025102,0.004184,0.004184,0.004184,0.008367,0.004184,0.004184,0.004184,...,0.004184,0.03347,0.012551,0.008367,0.008367,0.020918,0.004184,0.004184,0.004184,0.004184


,03,073,10,100,100000,1025,10k,10q,10th,11,...,ﬁscal,ﬁtness,ﬁve,ﬁx,ﬂat,ﬂight,ﬂights,ﬂow,ﬂuctuate,ﬂy
Transcript (Company & Date),,,,,,,,,,,,,,,,,,,,,
APPL 20181101,2,1,6,6,1,1,2,1,1,1,...,12,1,1,1,2,1,1,4,1,1


...

,082,10,100,100000,10k,10q,11,110,112,12,...,ﬁrst,ﬁrstinclass,ﬁrstparty,ﬁscal,ﬁve,ﬁveyear,ﬂat,ﬂeet,ﬂexibility,ﬂow
Transcript (Company & Date),,,,,,,,,,,,,,,,,,,,,
APPL 20200430,0.004378,0.004378,0.013133,0.004378,0.004378,0.004378,0.026266,0.004378,0.004378,0.008755,...,0.035021,0.004378,0.004378,0.008755,0.030644,0.008755,0.004378,0.004378,0.004378,0.008755


,022,10,100,10k,11,113,1142,115,12,122,...,ﬁnd,ﬁrst,ﬁtness,ﬁve,ﬂat,ﬂattish,ﬂood,ﬂow,ﬂows,ﬂuid
Transcript (Company & Date),,,,,,,,,,,,,,,,,,,,,
APPL 20210727,1,2,2,1,1,1,1,1,13,1,...,1,11,1,1,2,1,1,1,1,1


,022,10,100,10k,11,113,1142,115,12,122,...,ﬁnd,ﬁrst,ﬁtness,ﬁve,ﬂat,ﬂattish,ﬂood,ﬂow,ﬂows,ﬂuid
Transcript (Company & Date),,,,,,,,,,,,,,,,,,,,,
APPL 20210727,0.003508,0.007015,0.007015,0.003508,0.003508,0.003508,0.003508,0.003508,0.045598,0.003508,...,0.003508,0.038583,0.003508,0.003508,0.007015,0.003508,0.003508,0.003508,0.003508,0.003508


,063,10,100,1000,1000plus,103,104,107,10k,10q,...,ﬁnancial,ﬁnd,ﬁnds,ﬁnish,ﬁred,ﬁring,ﬁrst,ﬁscal,ﬁve,ﬂow
Transcript (Company & Date),,,,,,,,,,,,,,,,,,,,,
APPL 20171102,1,3,3,2,1,1,1,1,1,1,...,1,4,1,1,1,1,14,11,1,1


,063,10,100,1000,1000plus,103,104,107,10k,10q,...,ﬁnancial,ﬁnd,ﬁnds,ﬁnish,ﬁred,ﬁring,ﬁrst,ﬁscal,ﬁve,ﬂow
Transcript (Company & Date),,,,,,,,,,,,,,,,,,,,,
APPL 20171102,0.003993,0.011978,0.011978,0.007985,0.003993,0.003993,0.003993,0.003993,0.003993,0.003993,...,0.003993,0.01597,0.003993,0.003993,0.003993,0.003993,0.055896,0.043919,0.003993,0.003993
